# Boss Mobbing

## Description

When agents want to mob their boss by repeatedly performing poorly as a team.

In [190]:
import pandas as pd
import numpy as np
import altair as alt
import networkx as nx
alt.data_transformers.enable('vegafusion')

DataTransformerRegistry.enable('vegafusion')

In [191]:
log_name = 'propr'

fn_logs = {
    'bpic15': '../data/preproc/bpic15.csv',
    'bpic17': '../data/preproc/bpic17.csv',
    'propr': '../data/preproc/proprietary.csv'
}

In [192]:
log = pd.read_csv(fn_logs[log_name])
log

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,"School holidays; 0 = no, 1 = yes",Approval; 1 = low sum to 4 = high sum,Type of loan; 0 = normal; 1 = special,Cost
0,Approval_Branch,005141EEB1240B31C12577DF004F6A77,18/11/2010 15:27,18/11/2010 15:58,004-9-1,Thursday,0.0,2.0,0.0,"0,87 € per minute"
1,Precheck,005141EEB1240B31C12577DF004F6A77,19/11/2010 12:45,19/11/2010 12:46,000-3-01,Friday,0.0,2.0,0.0,"0,87 € per minute"
2,Precheck,005141EEB1240B31C12577DF004F6A77,24/11/2010 8:18,24/11/2010 8:26,000-2-01,Wednesday,0.0,2.0,0.0,"0,87 € per minute"
3,Check_of_Processing_Applications,005141EEB1240B31C12577DF004F6A77,24/11/2010 10:35,24/11/2010 10:35,000-2-01,Wednesday,0.0,2.0,0.0,"0,87 € per minute"
4,Processing_of_Applications,005141EEB1240B31C12577DF004F6A77,2/12/2010 10:46,2/12/2010 10:46,010-23-11,Thursday,0.0,2.0,0.0,"1,02 € per minute"
...,...,...,...,...,...,...,...,...,...,...
18440,Precheck,FFFF329EF772D73CC12577EA00534A1C,13/12/2010 11:38,13/12/2010 11:40,000-2-01,Monday,0.0,2.0,0.0,"0,87 € per minute"
18441,Check_of_Processing_Applications,FFFF329EF772D73CC12577EA00534A1C,13/12/2010 12:06,13/12/2010 12:14,000-2-01,Monday,0.0,2.0,0.0,"0,87 € per minute"
18442,Processing_of_Applications,FFFF329EF772D73CC12577EA00534A1C,22/12/2010 11:48,22/12/2010 11:48,010-23-07,Wednesday,1.0,2.0,0.0,"1,02 € per minute"
18443,Archieving,FFFF329EF772D73CC12577EA00534A1C,22/12/2010 11:49,22/12/2010 13:19,010-23-07,Wednesday,1.0,2.0,0.0,"1,02 € per minute"


In [193]:
# proprietary
log['department'] = log['resource'].apply(
    lambda x: x.split('-')[0]
)
log['department_role'] = log['resource'].apply(
    lambda x: x.split('-')[0] + '-' + x.split('-')[1]
)
log

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,"School holidays; 0 = no, 1 = yes",Approval; 1 = low sum to 4 = high sum,Type of loan; 0 = normal; 1 = special,Cost,department,department_role
0,Approval_Branch,005141EEB1240B31C12577DF004F6A77,18/11/2010 15:27,18/11/2010 15:58,004-9-1,Thursday,0.0,2.0,0.0,"0,87 € per minute",004,004-9
1,Precheck,005141EEB1240B31C12577DF004F6A77,19/11/2010 12:45,19/11/2010 12:46,000-3-01,Friday,0.0,2.0,0.0,"0,87 € per minute",000,000-3
2,Precheck,005141EEB1240B31C12577DF004F6A77,24/11/2010 8:18,24/11/2010 8:26,000-2-01,Wednesday,0.0,2.0,0.0,"0,87 € per minute",000,000-2
3,Check_of_Processing_Applications,005141EEB1240B31C12577DF004F6A77,24/11/2010 10:35,24/11/2010 10:35,000-2-01,Wednesday,0.0,2.0,0.0,"0,87 € per minute",000,000-2
4,Processing_of_Applications,005141EEB1240B31C12577DF004F6A77,2/12/2010 10:46,2/12/2010 10:46,010-23-11,Thursday,0.0,2.0,0.0,"1,02 € per minute",010,010-23
...,...,...,...,...,...,...,...,...,...,...,...,...
18440,Precheck,FFFF329EF772D73CC12577EA00534A1C,13/12/2010 11:38,13/12/2010 11:40,000-2-01,Monday,0.0,2.0,0.0,"0,87 € per minute",000,000-2
18441,Check_of_Processing_Applications,FFFF329EF772D73CC12577EA00534A1C,13/12/2010 12:06,13/12/2010 12:14,000-2-01,Monday,0.0,2.0,0.0,"0,87 € per minute",000,000-2
18442,Processing_of_Applications,FFFF329EF772D73CC12577EA00534A1C,22/12/2010 11:48,22/12/2010 11:48,010-23-07,Wednesday,1.0,2.0,0.0,"1,02 € per minute",010,010-23
18443,Archieving,FFFF329EF772D73CC12577EA00534A1C,22/12/2010 11:49,22/12/2010 13:19,010-23-07,Wednesday,1.0,2.0,0.0,"1,02 € per minute",010,010-23


## Social-SB-1

In [194]:
# convert to timestamps to calculate case/activity duration

log = log.sort_values(by='start timestamp')
log['start timestamp'] = pd.to_datetime(log['start timestamp'], format='mixed', dayfirst=True)
log['complete timestamp'] = pd.to_datetime(log['complete timestamp'], format='mixed', dayfirst=True)
log['activity_duration_minutes'] = (
    log['complete timestamp'] - log['start timestamp']
).dt.total_seconds() / 60

### Construct resource handover-of-work (HOW) network

In [195]:
all_resources = sorted(log['resource'].unique())
how = np.zeros((len(all_resources), len(all_resources)))
log = log.sort_values(by='complete timestamp')
for case_id, events in log.groupby('case_id'):
    res_seq = list(events['resource'])
    for i, r in enumerate(res_seq):
        px = all_resources.index(r)
        # ignore if the last resource in case
        if i + 1 == len(res_seq):
            pass
        else:
            r_next = res_seq[i+1]
            py = all_resources.index(r_next)
            how[px,py] += 1
how = pd.DataFrame.from_dict(how)
how['resource'] = all_resources
how = how.set_index('resource')
how = how.rename(columns=dict(
    (k, all_resources[k])
    for k in range(len(all_resources))
))
print(sum((how > 0).sum()))
how = nx.DiGraph(how)
print(how)
nx.write_graphml(how, fn_logs[log_name] + '.how.graphml')

1746
DiGraph with 220 nodes and 1746 edges


### Apply community detection to determine resource groups (communities)

In [196]:
# communities = nx.community.louvain_communities(how, seed=0)

In [197]:
communities = [
    list(log.loc[log['department'] == g, 'resource'].unique())
    for g in log['department'].unique()
]

### Determine the identities of "boss" resources in each resource group

In [198]:
sel_rgs = dict()
sel_rgs_boss = dict()
for ic, c in enumerate(communities):
    # consider only group with more than 3 members
    # consider group where the resource with the highest centrality is
    # significantly higher than all others
    print('Community {}: '.format(ic))
    if len(c) >= 3:
        sub_how = nx.subgraph(how, c)
        print(c)
        centrality_stats = pd.DataFrame.from_dict(
            # Coordinator: measured using betweenness centrality
            # nx.eigenvector_centrality(sub_how, max_iter=10000), 
            nx.betweenness_centrality(sub_how), 
            orient='index'
        ).reset_index().rename(columns={
            'index': 'resource',
            0: 'measured_value'
        }).sort_values(
            by='measured_value', ascending=False
        )
        ub = (
            centrality_stats['measured_value'].quantile(0.75) +
            1.5 * (centrality_stats['measured_value'].quantile(0.75) -
            centrality_stats['measured_value'].quantile(0.25))
        )
        if centrality_stats.iloc[0]['measured_value'] > ub:
            print(ub)
            print(centrality_stats)
            sel_rgs[ic] = c
            sel_rgs_boss[ic] = centrality_stats.iloc[0]['resource']
print(len(sel_rgs))
print(sel_rgs_boss)

Community 0: 
['020-10-01', '020-8-3', '020-9-1', '020-11-01', '020-10-02', '020-10-03', '020-19-5']
Community 1: 
['010-8-1', '010-7-04', '010-23-11', '010-24-26', '010-24-01', '010-24-23', '010-24-19', '010-24-17', '010-23-02', '010-24-11', '010-30-01', '010-23-13', '010-23-06', '010-23-01', '010-8-5', '010-23-15', '010-24-04', '010-21-01', '010-25-01', '010-6-05', '010-23-07', '010-24-12', '010-24-13', '010-5-04', '010-7-01', '010-5-01', '010-24-02', '010-23-05', '010-11-02', '010-8-4', '010-9-1', '010-25-07', '010-29-01', '010-6-02', '010-25-03', '010-23-14', '010-10-01', '010-23-08', '010-7-03', '010-5-02', '010-20-02', '010-15-02', '010-23-04', '010-15-01', '010-23-17', '010-25-06', '010-25-02', '010-24-16', '010-22-1', '010-10-03', '010-24-03']
0.047873692775478496
     resource  measured_value
2   010-24-26        0.378043
3   010-23-13        0.179580
5   010-24-12        0.049182
30  010-23-11        0.045714
45  010-24-23        0.040827
50  010-23-02        0.040203
49  010

### Discover co-presence of resource-boss

- Direct co-presence: resources worked in the same case
- Indirect co-presence: working time of resources overlap

In [199]:
res_attendance = dict()
for res, events in log.groupby('resource'):
    res_attendance[res] = [
        pd.Interval(e['start timestamp'], e['complete timestamp'], closed='both')
        for i, e in events.iterrows()
    ]

In [200]:
# Calculate direct co-presence
if False:
    direct_coapp = []
    for case, events in log.groupby('case_id'):
        case_team = set(events['resource'].unique())
        for i, e in events.iterrows():
            direct_coapp.append((
                i, 
                ','.join(sorted(case_team.difference({e['resource']})))
            ))
    direct_coapp = pd.DataFrame(
        direct_coapp, columns=['index', 'direct_copresenting_resources']
    )
    direct_coapp = direct_coapp.set_index(keys='index', drop=True)
    log = pd.merge(log, direct_coapp, left_index=True, right_index=True)

In [201]:
# Calculate indirect co-presence:
if False:
    all_resources = set(log['resource'].unique())
    indirect_coapp = []
    for res, events in log.groupby('resource'):
        for i, e in events.iterrows():
            overlapped_resources = set()
            interval = pd.Interval(
                e['start timestamp'], e['complete timestamp'], closed='both'
            )
            for other in all_resources.difference({res}):
                for x in res_attendance[other]:
                    if interval.overlaps(x):
                        overlapped_resources.add(other)
                        break
            indirect_coapp.append((i, ','.join(sorted(overlapped_resources))))
    indirect_coapp = pd.DataFrame(
        indirect_coapp, columns=['index', 'indirect_copresenting_resources']
    )
    indirect_coapp = indirect_coapp.set_index(keys='index', drop=True)
    log = pd.merge(log, indirect_coapp, left_index=True, right_index=True)

In [202]:
# log.to_csv(fn_logs[log_name] + '.coapp.csv', index=False)
log = pd.read_csv(fn_logs[log_name] + '.coapp.csv')
log

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,"School holidays; 0 = no, 1 = yes",Approval; 1 = low sum to 4 = high sum,Type of loan; 0 = normal; 1 = special,Cost,activity_duration_minutes,direct_copresenting_resources,indirect_copresenting_resources
0,Preparation_Draft_Executive_Board,6C9B95FFEBE57A0DC125782A002A3229,2011-02-01 10:01:00,2011-02-01 10:02:00,001-17-02,Tuesday,0.0,4.0,0.0,"2,04 € per minute",1.0,"010-23-07,010-23-13,070-15-02","010-23-11,010-24-01,010-24-23,010-24-26,070-10..."
1,Administrative_Tasks,08BC7C266A535237C1257826002ADD6E,2011-02-01 10:02:00,2011-02-01 10:51:00,010-24-26,Tuesday,0.0,1.0,1.0,"1,13 € per minute",49.0,"001-9-01,010-24-23","000-2-01,001-17-02,010-23-11,010-24-01,010-24-..."
2,Processing_of_Applications_Sonderkredit,08BC7C266A535237C1257826002ADD6E,2011-02-01 10:02:00,2011-02-01 10:02:00,010-24-26,Tuesday,0.0,1.0,1.0,"1,13 € per minute",0.0,"001-9-01,010-24-23","001-17-02,010-23-11,010-24-01,010-24-23,070-10..."
3,End,14050B07C84527C5C1257824002E58A8,2011-02-01 10:03:00,2011-02-01 10:03:00,010-23-11,Tuesday,0.0,3.0,0.0,"1,02 € per minute",0.0,"000-2-01,000-3-01,010-25-01,110-8-3","001-17-02,010-24-01,010-24-23,010-24-26,050-8-..."
4,Approval_Branch,3C3A3DDEF08D2225C1257829004E319B,2011-02-01 10:03:00,2011-02-01 10:09:00,050-8-1,Tuesday,0.0,2.0,0.0,"0,87 € per minute",6.0,"000-3-01,010-23-02,010-23-06,010-23-13","000-2-01,001-17-02,010-23-11,010-24-01,010-24-..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18440,Precheck,AB750FD25DEABED1C12577F300518367,2010-12-09 09:26:00,2010-12-09 10:13:00,070-7-06,Thursday,0.0,1.0,0.0,"1,02 € per minute",47.0,"010-23-13,010-24-26","010-24-01,010-24-17,010-24-19,010-7-04,010-8-1..."
18441,Precheck,059B8D0F8B58DFDEC12577F4002FAB94,2010-12-09 09:40:00,2010-12-09 14:18:00,093-18-1,Thursday,0.0,1.0,0.0,"0,87 € per minute",278.0,"010-24-04,010-24-26","001-15-03,001-5-01,001-6-02,001-7-02,010-23-02..."
18442,Precheck,401C68F8E2854EA7C12577F400309E68,2010-12-09 09:51:00,2010-12-09 09:54:00,020-9-1,Thursday,0.0,1.0,0.0,"0,87 € per minute",3.0,"010-23-13,010-24-26","010-24-17,010-24-19,010-8-1,070-7-06,093-18-1"
18443,Second_vote_KfW,DB060AF533AF5D13C1257735002E3378,2010-12-09 09:55:00,2010-12-09 09:55:00,010-24-01,Thursday,0.0,1.0,0.0,"1,13 € per minute",0.0,"010-24-17,010-24-19,010-24-23","010-24-17,010-24-19,010-8-1,070-7-06,093-18-1"


In [203]:
# label events by resource groups (communities) and boss resources
for ic, c in sel_rgs.items():
    log.loc[log['resource'].isin(c), 'resource_group'] = ic
    log.loc[log['resource'].isin(c), 'resource_group_boss'] = sel_rgs_boss[ic]
log

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,"School holidays; 0 = no, 1 = yes",Approval; 1 = low sum to 4 = high sum,Type of loan; 0 = normal; 1 = special,Cost,activity_duration_minutes,direct_copresenting_resources,indirect_copresenting_resources,resource_group,resource_group_boss
0,Preparation_Draft_Executive_Board,6C9B95FFEBE57A0DC125782A002A3229,2011-02-01 10:01:00,2011-02-01 10:02:00,001-17-02,Tuesday,0.0,4.0,0.0,"2,04 € per minute",1.0,"010-23-07,010-23-13,070-15-02","010-23-11,010-24-01,010-24-23,010-24-26,070-10...",3.0,001-5-02
1,Administrative_Tasks,08BC7C266A535237C1257826002ADD6E,2011-02-01 10:02:00,2011-02-01 10:51:00,010-24-26,Tuesday,0.0,1.0,1.0,"1,13 € per minute",49.0,"001-9-01,010-24-23","000-2-01,001-17-02,010-23-11,010-24-01,010-24-...",1.0,010-24-26
2,Processing_of_Applications_Sonderkredit,08BC7C266A535237C1257826002ADD6E,2011-02-01 10:02:00,2011-02-01 10:02:00,010-24-26,Tuesday,0.0,1.0,1.0,"1,13 € per minute",0.0,"001-9-01,010-24-23","001-17-02,010-23-11,010-24-01,010-24-23,070-10...",1.0,010-24-26
3,End,14050B07C84527C5C1257824002E58A8,2011-02-01 10:03:00,2011-02-01 10:03:00,010-23-11,Tuesday,0.0,3.0,0.0,"1,02 € per minute",0.0,"000-2-01,000-3-01,010-25-01,110-8-3","001-17-02,010-24-01,010-24-23,010-24-26,050-8-...",1.0,010-24-26
4,Approval_Branch,3C3A3DDEF08D2225C1257829004E319B,2011-02-01 10:03:00,2011-02-01 10:09:00,050-8-1,Tuesday,0.0,2.0,0.0,"0,87 € per minute",6.0,"000-3-01,010-23-02,010-23-06,010-23-13","000-2-01,001-17-02,010-23-11,010-24-01,010-24-...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18440,Precheck,AB750FD25DEABED1C12577F300518367,2010-12-09 09:26:00,2010-12-09 10:13:00,070-7-06,Thursday,0.0,1.0,0.0,"1,02 € per minute",47.0,"010-23-13,010-24-26","010-24-01,010-24-17,010-24-19,010-7-04,010-8-1...",4.0,070-21-01
18441,Precheck,059B8D0F8B58DFDEC12577F4002FAB94,2010-12-09 09:40:00,2010-12-09 14:18:00,093-18-1,Thursday,0.0,1.0,0.0,"0,87 € per minute",278.0,"010-24-04,010-24-26","001-15-03,001-5-01,001-6-02,001-7-02,010-23-02...",NaN,NaN
18442,Precheck,401C68F8E2854EA7C12577F400309E68,2010-12-09 09:51:00,2010-12-09 09:54:00,020-9-1,Thursday,0.0,1.0,0.0,"0,87 € per minute",3.0,"010-23-13,010-24-26","010-24-17,010-24-19,010-8-1,070-7-06,093-18-1",NaN,NaN
18443,Second_vote_KfW,DB060AF533AF5D13C1257735002E3378,2010-12-09 09:55:00,2010-12-09 09:55:00,010-24-01,Thursday,0.0,1.0,0.0,"1,13 € per minute",0.0,"010-24-17,010-24-19,010-24-23","010-24-17,010-24-19,010-8-1,070-7-06,093-18-1",1.0,010-24-26


In [204]:
log = log.dropna(how='any')

In [205]:
# label events by direct/indirect co-presence with boss
def is_resource_in(r, str_resources):
    resources = str_resources.split(',')
    if len(resources) > 0:
        return r in resources
    return np.nan

log['direct_copresenting_boss'] = log.apply(
    lambda row: is_resource_in(
        row['resource_group_boss'], row['direct_copresenting_resources']
    ),
    axis=1
)
log['indirect_copresenting_boss'] = log.apply(
    lambda row: is_resource_in(
        row['resource_group_boss'], row['indirect_copresenting_resources']
    ),
    axis=1
)
log

C:\Users\yangj29\AppData\Local\Temp\ipykernel_27220\621649649.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  log['direct_copresenting_boss'] = log.apply(
C:\Users\yangj29\AppData\Local\Temp\ipykernel_27220\621649649.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  log['indirect_copresenting_boss'] = log.apply(


,activity,case_id,start timestamp,complete timestamp,resource,Weekday,"School holidays; 0 = no, 1 = yes",Approval; 1 = low sum to 4 = high sum,Type of loan; 0 = normal; 1 = special,Cost,activity_duration_minutes,direct_copresenting_resources,indirect_copresenting_resources,resource_group,resource_group_boss,direct_copresenting_boss,indirect_copresenting_boss
0,Preparation_Draft_Executive_Board,6C9B95FFEBE57A0DC125782A002A3229,2011-02-01 10:01:00,2011-02-01 10:02:00,001-17-02,Tuesday,0.0,4.0,0.0,"2,04 € per minute",1.0,"010-23-07,010-23-13,070-15-02","010-23-11,010-24-01,010-24-23,010-24-26,070-10...",3.0,001-5-02,False,False
1,Administrative_Tasks,08BC7C266A535237C1257826002ADD6E,2011-02-01 10:02:00,2011-02-01 10:51:00,010-24-26,Tuesday,0.0,1.0,1.0,"1,13 € per minute",49.0,"001-9-01,010-24-23","000-2-01,001-17-02,010-23-11,010-24-01,010-24-...",1.0,010-24-26,False,False
2,Processing_of_Applications_Sonderkredit,08BC7C266A535237C1257826002ADD6E,2011-02-01 10:02:00,2011-02-01 10:02:00,010-24-26,Tuesday,0.0,1.0,1.0,"1,13 € per minute",0.0,"001-9-01,010-24-23","001-17-02,010-23-11,010-24-01,010-24-23,070-10...",1.0,010-24-26,False,False
3,End,14050B07C84527C5C1257824002E58A8,2011-02-01 10:03:00,2011-02-01 10:03:00,010-23-11,Tuesday,0.0,3.0,0.0,"1,02 € per minute",0.0,"000-2-01,000-3-01,010-25-01,110-8-3","001-17-02,010-24-01,010-24-23,010-24-26,050-8-...",1.0,010-24-26,False,True
5,Approval_Executive_Board,6C9B95FFEBE57A0DC125782A002A3229,2011-02-01 10:03:00,2011-02-01 10:04:00,001-17-02,Tuesday,0.0,4.0,0.0,"2,04 € per minute",1.0,"010-23-07,010-23-13,070-15-02","010-23-11,010-24-01,010-24-23,010-24-26,050-8-...",3.0,001-5-02,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18437,Further_inquiry,2F7101CBAF031C86C12577F200373AAE,2010-12-09 09:20:00,2010-12-09 09:57:00,010-24-17,Thursday,0.0,3.0,0.0,"1,13 € per minute",37.0,"000-3-01,010-24-13,010-24-26,010-30-01,020-9-1","010-24-01,010-24-19,010-8-1,016-9-1,020-9-1,07...",1.0,010-24-26,True,False
18438,Precheck,4E6EF51954A7212EC12577F4002DD4BD,2010-12-09 09:20:00,2010-12-09 10:12:00,010-8-1,Thursday,0.0,2.0,0.0,"0,87 € per minute",52.0,"000-3-01,010-23-07,010-23-13","010-24-01,010-24-17,010-24-19,010-7-04,016-9-1...",1.0,010-24-26,False,False
18440,Precheck,AB750FD25DEABED1C12577F300518367,2010-12-09 09:26:00,2010-12-09 10:13:00,070-7-06,Thursday,0.0,1.0,0.0,"1,02 € per minute",47.0,"010-23-13,010-24-26","010-24-01,010-24-17,010-24-19,010-7-04,010-8-1...",4.0,070-21-01,False,False
18443,Second_vote_KfW,DB060AF533AF5D13C1257735002E3378,2010-12-09 09:55:00,2010-12-09 09:55:00,010-24-01,Thursday,0.0,1.0,0.0,"1,13 € per minute",0.0,"010-24-17,010-24-19,010-24-23","010-24-17,010-24-19,010-8-1,070-7-06,093-18-1",1.0,010-24-26,False,False


### Profile resource productivity

In [206]:
# determine the activity duration distribution stats
act_dur_stats = []
for act, events in log.groupby('activity'):
    act_dur_stats.append({
        'activity': act,
        'duration_q3': events['activity_duration_minutes'].quantile(0.75),
        'duration_q1': events['activity_duration_minutes'].quantile(0.25),
    })
act_dur_stats = pd.DataFrame(act_dur_stats)
act_dur_stats['duration_ub_outlier'] = (
    act_dur_stats['duration_q3'] + 
    1.5 * (act_dur_stats['duration_q3'] - act_dur_stats['duration_q1'])
)
act_dur_stats

,activity,duration_q3,duration_q1,duration_ub_outlier
0,Administrative_Tasks,34.75,5.00,79.375
1,Application_Processing_Branches,39.00,3.00,93.000
2,Approval,15.00,2.00,34.500
3,Approval_Branch,43.00,3.00,103.000
4,Approval_Executive_Board,2.00,1.00,3.500
5,Archieving,66.00,7.00,154.500
6,Check_of_Approval,16.50,2.00,38.250
7,Check_of_Documents,74.00,30.00,140.000
8,Check_of_Processing_Applications,12.00,2.00,27.000
9,End,0.00,0.00,0.000


In [207]:
log = log.merge(act_dur_stats, how='left', on='activity')
log['is_low_performance'] = (
    log['activity_duration_minutes'] > log['duration_ub_outlier']
)
log

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,"School holidays; 0 = no, 1 = yes",Approval; 1 = low sum to 4 = high sum,Type of loan; 0 = normal; 1 = special,Cost,...,direct_copresenting_resources,indirect_copresenting_resources,resource_group,resource_group_boss,direct_copresenting_boss,indirect_copresenting_boss,duration_q3,duration_q1,duration_ub_outlier,is_low_performance
0,Preparation_Draft_Executive_Board,6C9B95FFEBE57A0DC125782A002A3229,2011-02-01 10:01:00,2011-02-01 10:02:00,001-17-02,Tuesday,0.0,4.0,0.0,"2,04 € per minute",...,"010-23-07,010-23-13,070-15-02","010-23-11,010-24-01,010-24-23,010-24-26,070-10...",3.0,001-5-02,False,False,11.50,2.00,25.750,False
1,Administrative_Tasks,08BC7C266A535237C1257826002ADD6E,2011-02-01 10:02:00,2011-02-01 10:51:00,010-24-26,Tuesday,0.0,1.0,1.0,"1,13 € per minute",...,"001-9-01,010-24-23","000-2-01,001-17-02,010-23-11,010-24-01,010-24-...",1.0,010-24-26,False,False,34.75,5.00,79.375,False
2,Processing_of_Applications_Sonderkredit,08BC7C266A535237C1257826002ADD6E,2011-02-01 10:02:00,2011-02-01 10:02:00,010-24-26,Tuesday,0.0,1.0,1.0,"1,13 € per minute",...,"001-9-01,010-24-23","001-17-02,010-23-11,010-24-01,010-24-23,070-10...",1.0,010-24-26,False,False,1.00,0.00,2.500,False
3,End,14050B07C84527C5C1257824002E58A8,2011-02-01 10:03:00,2011-02-01 10:03:00,010-23-11,Tuesday,0.0,3.0,0.0,"1,02 € per minute",...,"000-2-01,000-3-01,010-25-01,110-8-3","001-17-02,010-24-01,010-24-23,010-24-26,050-8-...",1.0,010-24-26,False,True,0.00,0.00,0.000,False
4,Approval_Executive_Board,6C9B95FFEBE57A0DC125782A002A3229,2011-02-01 10:03:00,2011-02-01 10:04:00,001-17-02,Tuesday,0.0,4.0,0.0,"2,04 € per minute",...,"010-23-07,010-23-13,070-15-02","010-23-11,010-24-01,010-24-23,010-24-26,050-8-...",3.0,001-5-02,False,False,2.00,1.00,3.500,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14184,Further_inquiry,2F7101CBAF031C86C12577F200373AAE,2010-12-09 09:20:00,2010-12-09 09:57:00,010-24-17,Thursday,0.0,3.0,0.0,"1,13 € per minute",...,"000-3-01,010-24-13,010-24-26,010-30-01,020-9-1","010-24-01,010-24-19,010-8-1,016-9-1,020-9-1,07...",1.0,010-24-26,True,False,37.75,2.25,91.000,False
14185,Precheck,4E6EF51954A7212EC12577F4002DD4BD,2010-12-09 09:20:00,2010-12-09 10:12:00,010-8-1,Thursday,0.0,2.0,0.0,"0,87 € per minute",...,"000-3-01,010-23-07,010-23-13","010-24-01,010-24-17,010-24-19,010-7-04,016-9-1...",1.0,010-24-26,False,False,20.00,1.00,48.500,True
14186,Precheck,AB750FD25DEABED1C12577F300518367,2010-12-09 09:26:00,2010-12-09 10:13:00,070-7-06,Thursday,0.0,1.0,0.0,"1,02 € per minute",...,"010-23-13,010-24-26","010-24-01,010-24-17,010-24-19,010-7-04,010-8-1...",4.0,070-21-01,False,False,20.00,1.00,48.500,False
14187,Second_vote_KfW,DB060AF533AF5D13C1257735002E3378,2010-12-09 09:55:00,2010-12-09 09:55:00,010-24-01,Thursday,0.0,1.0,0.0,"1,13 € per minute",...,"010-24-17,010-24-19,010-24-23","010-24-17,010-24-19,010-8-1,070-7-06,093-18-1",1.0,010-24-26,False,False,27.25,1.00,66.625,False


### Find correlation between co-presence periods and decreased performance of resources

In [208]:
col_copresence = 'direct_copresenting_boss'
charts = []
for ic, c in sel_rgs.items():
    # calculate the total number of activities performed by resources with or
    # without bosses
    # excluding the boss resource itself
    data = log.loc[
        (log['resource_group'] == ic) & 
        (log['resource'] != sel_rgs_boss[ic])
    ].groupby(['resource', col_copresence]).agg(
        avg_act_duration=pd.NamedAgg('activity_duration_minutes', 'mean')
    ).reset_index()
    data['productivity'] = 1 / data['avg_act_duration']

    cnt_mobbing_members = 0
    number_common_members = data['resource'].nunique()
    for r in data['resource'].unique():
        if (
            len(data.loc[
                (data['resource'] == r) & (data[col_copresence] == False),
                'productivity'
            ]) > 0 
            and
            len(data.loc[
                (data['resource'] == r) & (data[col_copresence] == True),
                'productivity'
            ]) > 0
        ):
            has_better_perf_without_boss = (
                data.loc[
                    (data['resource'] == r) & (data[col_copresence] == False),
                    'productivity'
                ].iloc[0] != np.inf
            ) and (
                data.loc[
                    (data['resource'] == r) & (data[col_copresence] == False),
                    'productivity'
                ].iloc[0] >
                data.loc[
                    (data['resource'] == r) & (data[col_copresence] == True),
                    'productivity'
                ].iloc[0]
            )
            if has_better_perf_without_boss:
                cnt_mobbing_members += 1
    subtitle = ('{} of {} members performed better in absence of boss'.format(
        cnt_mobbing_members, number_common_members
    ))
    data[col_copresence] = data[col_copresence].apply(
        lambda x: 'with boss' if x else 'without boss'
    )

    charts.append(
        alt.Chart(
            data, 
            title=[
                'Group (community) {}: Productivity in presence/absence of boss (resource {})'.format(ic, sel_rgs_boss[ic]),
                subtitle
            ]
        ).mark_bar().encode(
            x='resource:N',
            y='productivity:Q',
            column=alt.Column(f'{col_copresence}:N').title(None),
            tooltip=['resource', 'productivity']
        )
            
    )
alt.vconcat(*charts)

alt.VConcatChart(...)